In [1]:
import geoplotlib
from geoplotlib.utils import read_csv

In [2]:
import pandas as pd
import numpy as np

In [3]:
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [4]:
dataset = pd.read_csv('D:/Data/world_cities_pop.csv')

C:\Users\86137\AppData\Local\Temp\ipykernel_17780\3947710181.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('D:/Data/world_cities_pop.csv')


In [5]:
dataset.head(5)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6.0,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,6.0,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,6.0,NaN,42.466667,1.500000
3,ad,aixirvall,Aixirvall,6.0,NaN,42.466667,1.500000
4,ad,aixovall,Aixovall,6.0,NaN,42.466667,1.483333


In [6]:
dataset = dataset.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})

In [7]:
europe_country_codes = ['al', 'ad', 'at', 'by', 'be', 'ba', 'bg', 'hr', 'cy', 'cz', 'dk', 'ee', 'fo', 'fi', 'fr', 'de'
                        , 'gi', 'gr', 'hu', 'is', 'ie', 'im', 'it', 'xk', 'lv', 'li', 'lt', 'lu', 'mk', 'mt', 'md', 'mc'
                        , 'me', 'nl', 'no', 'pl', 'pt', 'ro', 'sm', 'rs', 'sk', 'si', 'es', 'se', 'ch', 'ua', 'gb'
                        , 'va']

In [8]:
filtered_dataset = dataset[dataset['Country'].isin(europe_country_codes)]

In [9]:
len(dataset)

3173958

In [10]:
len(filtered_dataset)

682348

In [11]:
z_cities = filtered_dataset[filtered_dataset['AccentCity'].str.startswith('Z',na=False)]

In [12]:
len(z_cities)

13218

In [13]:
z_cities.head(5)

,Country,City,AccentCity,Region,Population,lat,lon
104206,al,zaane,ZÄane,44.0,NaN,40.932778,19.783056
104207,al,zabarzani,Zabarzani,40.0,NaN,40.427778,20.269167
104208,al,zabarzan,Zabarzan,40.0,NaN,40.427778,20.269167
104209,al,zaberzane,Zabërzanë,40.0,NaN,40.427778,20.269167
104210,al,zaberzan i siperm,Zaberzan i Sipërm,40.0,NaN,40.427778,20.269167


In [14]:
from geoplotlib.utils import DataAccessObject
geoplotlib_data = DataAccessObject(z_cities)
geoplotlib.dot(geoplotlib_data, f_tooltip=lambda d: '{} - {}'.format(d['Country'].upper(), d['City']).title())
geoplotlib.show()

In [15]:
geoplotlib.voronoi(z_cities, cmap='Reds_r', alpha=50)
geoplotlib.show()

In [16]:
start_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'H', 'K', 'L'
                , 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W'
                , 'X', 'Y', 'Z']

In [32]:
import pyglet
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox
class FilterLayer (BaseLayer):
    def __init__(self, dataset, bbox=BoundingBox.WORLD):
        self.data = dataset
        self.start_letter = 0
        self.view = bbox

In [33]:
def invalidate(self, proj):
        start_letter_data =  self.data[self.data['AccentCity'].str.startswith(start_letters[self.start_letter])]
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(start_letter_data['lon'], start_letter_data['lat'])
        self.painter.points(x, y, 2)

In [34]:
def draw(self, proj, mouse_x, mouse_y, ui_manager):  
        self.painter.batch_draw()
        ui_manager.info('Displaying cities starting with {}'.format(start_letters[self.start_letter]))

In [39]:
def on_key_release(self, key, modifiers):
        if key == pyglet.window.key.RIGHT:
            self.start_letter = (self.start_letter + 1) % len(start_letters)
            return True
        elif key == pyglet.window.key.LEFT:
            self.start_letter = (self.start_letter - 1) % len(start_letters)
            return True
            
        return False
def bbox(self):
    return self.view

In [40]:
europe_bbox = BoundingBox(north=68.574309, west=-25.298424, south=34.266013, east=47.387123)
geoplotlib.add_layer(FilterLayer(filtered_dataset, europe_bbox))
geoplotlib.show()